In [22]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = 'Y6ftVJqnQh3iOJslwZ9_gZCsOYT7rZq52bWZ_PLxWD-3'
import time

In [23]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2020-04-09', 
                             authenticator=authenticator)

import json
with open('Motor.zip', 'rb') as Motor, \
     open('PP.zip', 'rb') as PowerPlant, \
     open('SS.zip', 'rb') as Substation, \
     open('Transformer.zip', 'rb') as Transformer, \
     open('Transmission Lines.zip', 'rb') as TransmissionLine:
        response = visrec.create_classifier(name="Device Classifier",
                                        positive_examples={'Motor': Motor, \
                                                           'Power plant': PowerPlant, \
                                                           'Substation': Substation, \
                                                           'Transformer': Transformer, \
                                                           'Transmission line': TransmissionLine})
print(json.dumps(response.get_result(), indent=2))

In [24]:
#lets grab the classifier id
classifier_id = 'DeviceClassifier_137907313'
classifier_id

'DeviceClassifier_137907313'

Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

print(time.ctime())
while(True):
    Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
    if Status!='training': 
        print(time.ctime())
        break

In [25]:
visrec.list_classifiers(verbose=True).get_result()

ERROR:root:Forbidden
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 157, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403


ApiException: Error: Forbidden, Code: 403

In [20]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

In [21]:
getdf_visrec(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR8Vr4SxclbQeRXjZFUaZk4qN2icFe5o_9jgp1qigezPlYw7Frs&usqp=CAU',
            classifier_ids=classifier_id)

ERROR:root:Forbidden
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 157, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403


ApiException: Error: Forbidden, Code: 403

In [ ]:
import wget, zipfile, os

filename = wget.download('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/CV0101/Project/CV0101EN-capstone-project-new.zip')
with zipfile.ZipFile(filename,"r") as zip_ref:
    zip_ref.extractall()